# Tesla specific time series processing
The goal of this notebook is to demonstrate the implementation of time series processing steps that are specific to Tesla.

## Setup

### Imports

In [ ]:
import plotly.express as px

from pandas.api.types import CategoricalDtype

from core.pandas_utils import *
from transform.processed_tss.config import IN_CHARGE_CHARGING_STATUS_VALS, IN_DISCHARGE_CHARGING_STATUS_VALS
from transform.processed_tss.ProcessedTimeSeries import TeslaProcessedTimeSeries
from transform.raw_results.tesla_results import get_results

### Data extraction

In [ ]:
! mkdir -p data_cache

In [ ]:
tss = TeslaProcessedTimeSeries()

In [ ]:
tss = tss.astype({
    "vin": CategoricalDtype(),
    "charging_status": CategoricalDtype(),
    "charging_method": CategoricalDtype(),
})

In [ ]:
TARGET_VIN = "LRW3E7FA4MC314534"

## Charging masksing and indexing

In [ ]:
def compute_charge_mask(tss:DF) -> DF:
    base = (
        Series(pd.NA, index=tss.index, dtype="boolean")
        .mask(tss["charging_status"].isin(IN_CHARGE_CHARGING_STATUS_VALS), True)
        .mask(tss["charging_status"].isin(IN_DISCHARGE_CHARGING_STATUS_VALS), False)
    )
    ffill_base = base.groupby(tss["vin"], observed=False).ffill()
    bfill_base = base.groupby(tss["vin"], observed=False).bfill()
    base = base.mask(ffill_base.eq(bfill_base), ffill_base)
    base = base.mask(tss["soc"] >= 98)
    tss["in_charge"] = base
    return tss

def compute_charge_idx(tss:DF) -> DF:
    tss_grp = tss.groupby("vin", observed=False)
    tss["charge_energy_added"] = tss_grp["charge_energy_added"].ffill()
    power_loss = tss_grp['charge_energy_added'].diff().div(tss["sec_time_diff"])
    min_power_loss = (
        power_loss
        .loc[tss["charging_status"] == 'stopped']
        .quantile(0.05)
    )
    new_charge_mask = power_loss.lt(min_power_loss, fill_value=0)
    tss["charge_idx"] = new_charge_mask.groupby(tss["vin"], observed=False).cumsum()
    return tss

def compute_status_col(tss:DF) -> DF:
    tss_grp = tss.groupby("vin", observed=False)
    status = tss["in_charge"].map({True: "charging", False:"discharging", pd.NA:"unknown"})
    tss["status"] = status.mask(
        tss["in_charge"].eq(False, fill_value=True),
        np.where(tss_grp["odometer"].diff() > 0, "moving", "idle_discharging"),
    )

    return tss


In [ ]:
tss = (
    tss
    .pipe(compute_charge_mask)
    .pipe(compute_charge_idx)
    .pipe(compute_status_col)
)

In [ ]:
ts = tss.query("vin == @TARGET_VIN").copy()

In [ ]:
px.scatter(
    (
        ts
        #.eval("in_charge_str = in_charge.astype('string').fillna('unknown')")
        .melt(['odometer', "date", "charge_idx", "status"], ["soc"])
    ),
    x="date",
    y="value",
    facet_row="variable",
    color="status",
    hover_data=["odometer"]
).update_yaxes(matches=None)

In [ ]:
THIBAULT_VIN = "5YJ3E7EB7KF474436"
thibault_ts = tss.query("vin == @THIBAULT_VIN")
thibault_ts["charge_limit_soc"].value_counts(dropna=False)

In [ ]:
thibault_ts:DF = (
    thibault_ts
    .eval("in_charge_below_98 = in_charge & soc < 98")
    .pipe(compute_in_charge_idx)
    .pipe(trim_leading_n_trailing_soc_off_masks, ["in_charge"])
    .pipe(fill_uncertain_charge_periods, "in_charge_below_98")
    .pipe(compute_idx_from_masks, ["trimmed_in_charge"], TD(hours=1, minutes=30))
)

In [ ]:
display(thibault_ts.query("nan_charge == True")["nan_charge_idx"].nunique())
display(thibault_ts.query("trimmed_in_charge")["trimmed_in_charge_idx"].nunique())

In [ ]:
px.scatter(
    thibault_ts.eval("in_charge_str = nan_charge.astype('string').fillna('unknown')"),
    x="date",
    y="soc",
    color="in_charge_str",
    symbol="nan_charge_idx",
    hover_data=[
        "in_charge_str",
        "trimmed_in_charge",
        "sec_time_diff",
        "charge_energy_added",
        "charging_status",
        "leading_soc",
        "trailing_soc",
    ],
).update_layout(showlegend=True)

In [ ]:
px.scatter(
    thibault_ts.eval("in_charge_str = nan_charge.astype('string').fillna('unknown')"),
    x="date",
    y="soc",
    color="trimmed_in_charge",
    symbol="trimmed_in_charge_idx",
    hover_data=[
        "in_charge_str",
        "trimmed_in_charge",
        "sec_time_diff",
        "charge_energy_added",
        "charging_status",
        "leading_soc",
        "trailing_soc",
    ],
).update_layout(showlegend=True)

In [ ]:
thibault_ts.melt(id_vars=["vin", "date"], value_vars=["soc", "charge_limit_soc"])

In [ ]:
thibault_ts["charge_limit_soc_std"].value_counts()

In [ ]:
thibault_ts.dtypes

In [ ]:
px.scatter(
    (
        thibault_ts
        .eval(" = odometer")
        .eval("test = charge_limit_soc + charge_limit_soc_std / 10")
        .melt(id_vars=["vin", "date"], value_vars=["soc", "test", "odometer"])
    ),
    x="date",
    y="value",
    color="variable"
)

In [ ]:
tss.dtypes